# <i> Multiple Pooling Convolutional Neural Network 

In [1]:
from keras.preprocessing.image import ImageDataGenerator
class Datagenerator:
    def __init__(self, train_path, test_path):
        self.train_path = train_path 
        self.test_path = test_path 
        
    def generate_train(self):
        train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
        training_set = train_datagen.flow_from_directory(self.train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
        return training_set 
    def generate_test(self):
        test_datagen = ImageDataGenerator(rescale = 1./255)
        test_set = test_datagen.flow_from_directory(self.test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')
        return test_set 

In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf 

class Cnn_Model():
    def __init__(self, size, channels):
        self.size = size
        self.channels = channels
        self.input_ = None
        self.model = None
        self.input_shape = (self.size, self.size, self.channels)

    def build(self):
        self.input_ = keras.layers.Input(self.input_shape)

        # first block
        conv1 = keras.layers.Conv2D(
            32, (3, 3), activation='relu', padding='same')(self.input_)
        
        # multiple pooling part  ------> 
        
        # pooling on the right side 
        pool_max = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
        
        # getting pooling on the left side 
        nega = tf.math.negative(conv1) 
        nega_pool = keras.layers.MaxPooling2D(pool_size = (2,2))(nega) 
        # taking negativ again 
        pool_range = tf.math.negative(nega_pool) 
        # gettting the range pooling 
        range_pool = tf.math.subtract(pool_max,pool_range)
        
        # concatenating pools 
        concatenated = tf.keras.layers.Concatenate()([pool_max, range_pool]) 
        
        # -----------> 
        
        
        norm1 = keras.layers.BatchNormalization(axis=-1)(concatenated)
        drop1 = keras.layers.Dropout(rate=0.2)(norm1)  # adding regularization


        # flattening weights
        flatten = keras.layers.Flatten()(drop1)

        # fully connected set 1
        hidden1 = keras.layers.Dense(512, activation='relu')(flatten)
        norm3 = keras.layers.BatchNormalization(axis=-1)(hidden1)
        drop3 = keras.layers.Dropout(rate=0.2)(norm3)

        # fully connected set2
        hidden2 = keras.layers.Dense(512, activation='relu')(drop3)
        norm4 = keras.layers.BatchNormalization(axis=-1)(hidden2)
        drop4 = keras.layers.Dropout(rate=0.2)(norm4)

        # output layer
        self.output_ = keras.layers.Dense(6, activation='softmax')(drop4)

    def compile(self):
        # model compiling
        self.model = keras.Model(inputs=self.input_, outputs=self.output_)
        self.model.compile(optimizer='adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])

    # fit the model
    def fit_generator(self, epochs,train_data,test_data ):
        return self.model.fit(
            train_data,
            validation_data=test_data,
            epochs=epochs,
            steps_per_epoch=len(training_set),
             validation_steps=len(test_set)
        )

    def get_summary(self):
        return self.model.summary()

In [ ]:
# pass the training and testing path 
training_path  =' '
testing_path = ' '
loader = Datagenerator(training_path, testing_path)
training_set = loader.generate_train() 
testing_set = loader.generate_test() 

model = Cnn_Model(224,3)
model.build()
model.compile()

print(model.get_summary()) 

# training part 
model.fit_generator(epochs = 2,train_data = training_set, test_data = test_set)